In [0]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.3'
spark_version = 'spark-3.5.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://repos.azul.com/zulu/deb stable InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Hit:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4337 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4699 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [4526 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1298 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [4223 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1590 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 P

In [0]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, col, round, avg
from pyspark.sql.types import StringType
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [0]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [0]:
# Upload the file to DBFS
dbutils.fs.cp("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv", "dbfs:/FileStore/tables/home_sales_revised.csv")


Out[7]: True

In [0]:
# Read the home_sales_revised.csv from DBFS into a PySpark DataFrame
file_location = "dbfs:/FileStore/tables/home_sales_revised.csv"
home_sales_df = spark.read.option("header", "true").option("inferSchema", "true").csv(file_location)


In [0]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")

In [0]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

# Run the query using PySpark
four_bedroom_avg_price_df = home_sales_df.filter(home_sales_df.bedrooms == 4) \
    .groupBy(year(home_sales_df.date).alias("year")) \
    .agg(round(avg("price"), 2).alias("avg_price")) \
    .orderBy("year")

# Show the results
four_bedroom_avg_price_df.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [0]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
home_sales_df = home_sales_df.withColumn("date_built_str", col("date_built").cast(StringType()))

avg_price_df = home_sales_df.filter((home_sales_df.bedrooms == 3) & (home_sales_df.bathrooms == 3)) \
    .groupBy(home_sales_df.date_built_str.alias("year_built")) \
    .agg(round(avg("price"), 2).alias("avg_price")) \
    .orderBy("year_built")

avg_price_df.show()                                                                  

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [0]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
filtered_df = home_sales_df.filter(
    (home_sales_df.bedrooms == 3) &
    (home_sales_df.bathrooms == 3) &
    (home_sales_df.floors == 2) &
    (home_sales_df.sqft_living >= 2000)
)

avg_price_df = filtered_df.groupBy(col("date_built").alias("year_built")) \
    .agg(round(avg("price"), 2).alias("avg_price")) \
    .orderBy("year_built")

avg_price_df.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [0]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_avg_price_df = home_sales_df.groupBy("view") \
    .agg(round(avg("price"), 2).alias("avg_price")) \
    .filter("avg_price >= 350000") \
    .orderBy(col("view").desc())

view_avg_price_df.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.4741864204406738 seconds ---


In [0]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

Out[25]: DataFrame[]

In [0]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

Out[26]: True

In [0]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

view_avg_price_df = spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
""")

view_avg_price_df.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.3503689765930176 seconds ---


In [0]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").parquet("/path/to/partitioned_home_sales")

In [0]:
# 11. Read the parquet formatted data.
partitioned_df = spark.read.parquet("/path/to/partitioned_home_sales")
partitioned_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+--------------+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built_str|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+--------------+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|          2015|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|          2015|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|          2015|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|          2015|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|   

In [0]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView("partitioned_home_sales")

In [0]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

view_avg_price_parquet_df = spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM partitioned_home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
""")

view_avg_price_parquet_df.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 2.492353916168213 seconds ---


In [0]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

Out[32]: DataFrame[]

In [0]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales')
print(f"Is 'home_sales' table cached? {is_cached}")

Is 'home_sales' table cached? False
